In [1]:
from flask import Flask,request
import cv2
import numpy as np
import base64
from PIL import Image
from io import BytesIO


In [2]:
app = Flask(__name__)

In [3]:
# antwoord code van Nathancy
# https://stackoverflow.com/questions/58613825/get-boundary-from-canny-edges-and-remove-the-background-of-an-image
# geraadpleegd op 17/nomeber 2020 

def rb(file,n):
    npimg = np.fromfile(file, np.uint8)
    image = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
    image = image[:,:,::-1]
    height, width, channels = image.shape
    if channels < 4:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
    original = image.copy()
    mask = np.zeros(image.shape, dtype=np.uint8)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    thresh = cv2.threshold(gray,n, 255, cv2.THRESH_BINARY_INV)[1]
    if np.any(thresh[0,0] != 0):
        thresh = cv2.threshold(gray,n, 255, cv2.THRESH_BINARY)[1]

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=3)
    
    cnts = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    for c in cnts:
        cv2.drawContours(mask, [c], -1, (255,255,255), -1)
        break

    close = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=4)
    close = cv2.cvtColor(close, cv2.COLOR_BGR2GRAY)
    result = cv2.bitwise_and(original, original, mask=close)
    result[close==0] = (0,0,0,0)

    cnt = sorted(cnts, key=cv2.contourArea)[-1]
    x,y,w,h = cv2.boundingRect(cnt)
    final = result[y:y+h, x:x+w]

# antwoord code van Eugene V
# https://stackoverflow.com/questions/31826335/how-to-convert-pil-image-image-object-to-base64-string
# geraadpleegd op 24 Februari 2021
    
    pil_image = Image.fromarray(final.astype('uint8'), 'RGBA')
    buffered = BytesIO()
    pil_image.save(buffered, format="PNG")
    encoded_img = base64.encodebytes(buffered.getvalue()).decode('ascii') # encode as base64
    #print(encoded_img)
    return encoded_img


In [4]:
@app.route('/img',methods = ['POST'])
def img():
    if request.method == 'POST':
        file = request.files.get("image")
        n = request.values.get("n")
        return str(rb(file,int(n)))
#    files = request.files.getlist("image")
#    for file in files:
#        imglist.append(str(rb(file)))
#    return json.dumps(imglist)


In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Aug/2021 12:38:58] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:39:02] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:39:30] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:39:31] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:40:50] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:40:51] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:40:53] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:40:54] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:40:55] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:40:58] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:40:59] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:41:00] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:41:54] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:41:55] "POST /img HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2021 12:41:56] "POST /